In [24]:
import Pkg
Pkg.activate(@__DIR__)

  Activating environment at `~/all_about_robotics/julia_optimisation_course/tutorial/Project.toml`


In [25]:
using LinearAlgebra
using Plots
import ForwardDiff as Fd

In [26]:
# define the feedback linearized model of the simple pendulum
m=1    # mass is of the pendulum
g=9.81 # gravity
l=1 # length of the pendulum

h=0.001 # time step for rk45 integrator

# feedback linearized dynamics
function dynamics(x,τ)
     x1,x2=x
    x1_dot= x2
    x2_dot= -(g / l)*sin(x1)+(τ / (m*l^2))
    return [x1_dot,x2_dot]
end
    

dynamics (generic function with 1 method)

In [27]:
function rk45step(x,τ_v)
f1=dynamics(x,τ_v)
   f2=dynamics(x+0.5*h*f1,τ_v)
   f3=dynamics(x+0.5*h*f2,τ_v)
   f4=dynamics(x+h*f3,τ_v)
    
   return x+(h/6)*(f1+2*f2+2*f3+f4)
    
end

rk45step (generic function with 1 method)

In [28]:
function rk45_solve(t0, x0, tf)
    t = t0
    x = x0
        
    ts = [t] # vector to save time steps
    xs = [x0] # vector to save state trajectory
    
    while t < tf
        x_new = rk45step(x,0)
        t += h
        x = x_new
        push!(ts, t)
        push!(xs, x)
    end
    
    return ts, xs
end

rk45_solve (generic function with 1 method)

In [29]:

x0=[pi-0.001, 0.0]

# Time span
t0 = 0.0
tf = 100.0
t_vals, x_vals = rk45_solve(t0,x0,tf)

# Extract angle values
θ_vals = [x[1] for x in x_vals]
w_vals=[x[2] for x in x_vals]
# Plot the results
plot(t_vals, θ_vals, label="Angle", xlabel="Time", ylabel="Angle", legend=true)
plot!(t_vals, w_vals, label="omega", xlabel="Time", ylabel="omega", legend=true,show=true)